# Project 

In [2]:
import mujoco
import mediapy as media
import numpy as np
import time
import mujoco.viewer as viewer

modelxml = """
<mujoco model="CartPole">
  <compiler eulerseq="XYZ"/>
  <default>
    <default class="unused"/>
  </default>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1="0.1 0.2 0.3" rgb2="0.2 0.3 0.4" width="512" height="512"/>
    <material name="grid" class="unused" texture="grid" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
  </asset>
  <worldbody>
    <geom name="floor" class="unused" type="plane" condim="3" size="0 0 0.05" material="grid" pos="0 0 -1"/>
    <body name="Cart" pos="0 0 0" euler="0 0 0">
      <!-- 小车的惯性设置为接近零 -->
      <inertial pos="0 0 0" mass="10" diaginertia="1e-11 1e-11 1e-11"/>
      <geom name="cart_visual" class="unused" type="box" contype="0" conaffinity="0" group="0" size="0.12 0.06 0.06" pos="0 0 0"/>
      <joint name="CartSlider" class="unused" type="slide" pos="0 0 0" axis="1 0 0"/>
      <body name="Pole" pos="0 0 -0.5" euler="0 0 0">
        <inertial pos="0 0 0" mass="1" diaginertia="1e-11 1e-11 1e-11"/>
        <geom name="pole_point_mass" class="unused" type="sphere" contype="0" conaffinity="0" group="0" size="0.05" pos="0 0 0"/>
        <geom name="pole_rod" class="unused" type="cylinder" contype="0" conaffinity="0" group="0" size="0.025 0.25" pos="0 0 0.25"/>
        <joint name="PolePin" class="unused" type="hinge" pos="0 0 0.5" axis="0  -1  0"/>
      </body>
    </body>
  </worldbody>
  <sensor>
        <!-- 关节位置传感器 -->
        <jointpos joint="CartSlider" name="cart_p"/>
        <jointpos joint="PolePin" name="pole_theta"/>
        <!-- 关节速度传感器 -->
        <jointvel joint="CartSlider" name="cart_v"/>
        <jointvel joint="PolePin" name="pole_w"/>
  </sensor>
  <actuator>
    <motor joint="CartSlider"/>
  </actuator>
  <keyframe>
    <!-- 这里的 keyframe 用于初始状态设置，不过我们之后会手动修改初始条件 -->
    <key name="off1" qpos="0 3.15"/>
  </keyframe>
</mujoco>
"""

# 创建模型和仿真数据
model = mujoco.MjModel.from_xml_string(modelxml)
data = mujoco.MjData(model)



# Part 4
Simulation Setup and Testing:

In [ ]:


def run_simulation(initial_qpos, initial_qvel, u=0, duration=10000):
    """
    根据给定的初始位置(initial_qpos)和初始速度(initial_qvel)运行仿真。
    u: 控制输入
    duration: 仿真持续时间（秒）
    """

    data.qpos[:] = initial_qpos
    data.qvel[:] = initial_qvel
    mujoco.mj_forward(model, data)
    
    with viewer.launch_passive(model, data) as sim_viewer:
        start_time = time.time()
        while sim_viewer.is_running():
            step_start = time.time()
            
            # 无控制作用，保持控制输入为零
            data.ctrl = u
            mujoco.mj_step(model, data)  
            sim_viewer.sync()  
            
            time_until_next = model.opt.timestep - (time.time() - step_start)
            if time_until_next > 0:
                time.sleep(time_until_next)

# 初始条件1：小车在 0 位置，杆为倒立位置（π 弧度），速度均为0
initial_qpos1 = np.array([0.0, np.pi/2])
initial_qvel1 = np.array([0.0, 0.0])
 
# 初始条件2：小车在 0 位置，杆有一定偏差（π+0.2 弧度），速度均为0
initial_qpos2 = np.array([0, np.pi + 0.2])
initial_qvel2 = np.array([0.0, 0.0])

print("Simulating uncontrolled cart-pole with initial condition set 1")
run_simulation(initial_qpos1, initial_qvel1, duration=10)

print("Simulating uncontrolled cart-pole with initial condition set 2")
run_simulation(initial_qpos2, initial_qvel2, u=0.5, duration=10)

Simulating uncontrolled cart-pole with initial condition set 1
Simulating uncontrolled cart-pole with initial condition set 2


# Part 5
    (a)

In [4]:
import numpy as np
import scipy.linalg

T = 0.002  # 采样时间
model.opt.timestep = T  

mc = 10
mp = 1
L = 0.5
g = 9.81

Ac = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, -1],
    [0, -mp*g/mc, 0, 0],
    [0, -((mc+mp)*g)/(L*mc), 0, 0]
])
Bc = np.array([
    [0],
    [0],
    [1/mc],
    [1/(L*mc)]
])

A = np.eye(4) + Ac * T
B = Bc * T

# LQR 权重矩阵
Q = np.diag([100, 10, 1, 1])
R = np.diag([0.1])
print("Q:", Q)
print("R:", R)

def calculate_K(Q, R):
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)
    K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)
    return K

K = calculate_K(Q, R)
print("K:", K)


Q: [[100   0   0   0]
 [  0  10   0   0]
 [  0   0   1   0]
 [  0   0   0   1]]
R: [[0.1]]
K: [[ -31.25492485 -357.59207105  -40.04072029   78.5219108 ]]


# Part 5
    (b)

In [ ]:
import time
import numpy as np
import mujoco
import mujoco.viewer as viewer

# LQR控制器
def LQR_control(model, data, K):
    x1 = data.qpos[0]          # 小车位置 z
    x2 = np.pi - data.qpos[1]  # 角度偏差 π-θ
    x3 = data.qvel[0]          # 小车速度 ż
    x4 = data.qvel[1]          # 角速度 θ̇
    x = np.array([x1, x2, x3, x4])  # 状态向量
    u = - (K @ x).item()  
    return u

def run_closed_loop(initial_qpos, initial_qvel, K, duration=5):

    data.qpos[:] = initial_qpos
    data.qvel[:] = initial_qvel
    mujoco.mj_forward(model, data)  

    with viewer.launch_passive(model, data) as sim_viewer:
        start_time = time.time()
        while sim_viewer.is_running() and time.time() - start_time < duration:
            step_start = time.time()

            u = LQR_control(model, data, K)
            data.ctrl[:] = u

            mujoco.mj_step(model, data)
            sim_viewer.sync()  

            time_until_next = model.opt.timestep - (time.time() - step_start)
            if time_until_next > 0:
                time.sleep(time_until_next)

# 定义四种不同的初始状态
initial_conditions = [
    (np.array([0.0, np.pi - 0.2]), np.array([0.0, 0.0])),  
    (np.array([0.0, np.pi - 0.5]), np.array([0.0, 0.0])), 
    (np.array([0.0, np.pi - 0.5]), np.array([0.5, 1.0])),  
    (np.array([0.0, np.pi - 0.5]), np.array([1.0, 2.0])),  
]

# 运行闭环仿真
for init_qpos, init_qvel in initial_conditions:
    run_closed_loop(init_qpos, init_qvel, K, duration=10)



# Part 6

In [6]:
# Q 和 R 矩阵定义
Q1 = np.diag([10, 100, 1, 1])  
R1 = np.diag([0.1])

# 增加速度的权重
Q2 = np.diag([1, 10, 10, 10])  
R2 = np.diag([0.5])

# 平滑控制输入
Q3 = np.diag([5, 50, 5, 5]) 
R3 = np.diag([2])

# 快速响应设计
Q4 = np.diag([100, 1000, 10, 10])  
R4 = np.diag([0.1])


# 计算LQR增益矩阵
K1 = calculate_K(Q1, R1)
K2 = calculate_K(Q2, R2)
K3 = calculate_K(Q3, R3)
K4 = calculate_K(Q4, R4)

# 运行仿真
run_closed_loop(init_qpos, init_qvel, K1, duration=10)  
run_closed_loop(init_qpos, init_qvel, K2, duration=10)  
run_closed_loop(init_qpos, init_qvel, K3, duration=10)  
run_closed_loop(init_qpos, init_qvel, K4, duration=10)  